In [1]:
import numpy as np
from scipy import sparse

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.utils import check_array
from sklearn.utils.validation import check_is_fitted, _check_feature_names_in
from sklearn.preprocessing import OneHotEncoder
import sklearn
import sklearn.impute

import pandas as pd
from pandas.api.types import is_numeric_dtype
import sklearn.compose
from scipy import optimize

import torch
import sklearn.preprocessing
import openml
import tpot2
import sklearn.metrics
import sklearn
from sklearn.metrics import (roc_auc_score, roc_curve, precision_score, auc, recall_score, precision_recall_curve, \
                             roc_auc_score, accuracy_score, balanced_accuracy_score, f1_score, log_loss,
                             f1_score, root_mean_squared_error)


/Users/gabrielketron/tpot2_addimputers/env2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def add_missing(X, add_missing = 0.05, missing_type = 'MAR'):
    if isinstance(X,np.ndarray):
        X = pd.DataFrame(X)
    missing_mask = X
    missing_mask = missing_mask.mask(missing_mask.isna(), True)
    missing_mask = missing_mask.mask(missing_mask.notna(), False)
    X = X.mask(X.isna(), 0)
    T = torch.tensor(X.to_numpy())

    match missing_type:
        case 'MAR':
            out = MAR(T, [add_missing])
        case 'MCAR':
            out = MCAR(T, [add_missing])
        case 'MNAR':
            out = MNAR_mask_logistic(T, [add_missing])
    
    masked_set = pd.DataFrame(out['Mask'].numpy())
    missing_combo = (missing_mask | masked_set.isna())
    masked_set = masked_set.mask(missing_combo, True)
    masked_set.columns = X.columns.values
    #masked_set = masked_set.to_numpy()

    missing_set = pd.DataFrame(out['Missing'].numpy())
    missing_set.columns = X.columns.values
    #missing_set = missing_set.to_numpy()

    return missing_set, masked_set

"""BEYOND THIS POINT WRITTEN BY Aude Sportisse, Marine Le Morvan and Boris Muzellec - https://rmisstastic.netlify.app/how-to/python/generate_html/how%20to%20generate%20missing%20values"""

def MCAR(X, p_miss):
    out = {'X': X.double()}
    for p in p_miss: 
        mask = (torch.rand(X.shape) < p).double()
        X_nas = X.clone()
        X_nas[mask.bool()] = np.nan
        model_name = 'Missing'
        mask_name = 'Mask'
        out[model_name] = X_nas
        out[mask_name] = mask
    return out

def MAR(X,p_miss,p_obs=0.5):
    out = {'X': X.double()}
    for p in p_miss:
        n, d = X.shape
        mask = torch.zeros(n, d).bool()
        num_no_missing = max(int(p_obs * d), 1)
        num_missing = d - num_no_missing
        obs_samples = np.random.choice(d, num_no_missing, replace=False)
        copy_samples = np.array([i for i in range(d) if i not in obs_samples])
        len_obs = len(obs_samples)
        len_na = len(copy_samples)
        coeffs = torch.randn(len_obs, len_na).double()
        Wx = X[:, obs_samples].mm(coeffs)
        coeffs /= torch.std(Wx, 0, keepdim=True)
        coeffs.double()
        len_obs, len_na = coeffs.shape
        intercepts = torch.zeros(len_na)
        for j in range(len_na):
            def f(x):
                return torch.sigmoid(X[:, obs_samples].mv(coeffs[:, j]) + x).mean().item() - p
            intercepts[j] = optimize.bisect(f, -50, 50)
        ps = torch.sigmoid(X[:, obs_samples].mm(coeffs) + intercepts)
        ber = torch.rand(n, len_na)
        mask[:, copy_samples] = ber < ps
        X_nas = X.clone()
        X_nas[mask.bool()] = np.nan
        model_name = 'Missing'
        mask_name = 'Mask'
        out[model_name] = X_nas
        out[mask_name] = mask
    return out

def MNAR_mask_logistic(X, p_miss, p_params =.5, exclude_inputs=True):
    """
    Missing not at random mechanism with a logistic masking model. It implements two mechanisms:
    (i) Missing probabilities are selected with a logistic model, taking all variables as inputs. Hence, values that are
    inputs can also be missing.
    (ii) Variables are split into a set of intputs for a logistic model, and a set whose missing probabilities are
    determined by the logistic model. Then inputs are then masked MCAR (hence, missing values from the second set will
    depend on masked values.
    In either case, weights are random and the intercept is selected to attain the desired proportion of missing values.
    Parameters
    ----------
    X : torch.DoubleTensor or np.ndarray, shape (n, d)
        Data for which missing values will be simulated.
        If a numpy array is provided, it will be converted to a pytorch tensor.
    p : float
        Proportion of missing values to generate for variables which will have missing values.
    p_params : float
        Proportion of variables that will be used for the logistic masking model (only if exclude_inputs).
    exclude_inputs : boolean, default=True
        True: mechanism (ii) is used, False: (i)
    Returns
    -------
    mask : torch.BoolTensor or np.ndarray (depending on type of X)
        Mask of generated missing values (True if the value is missing).
    """
    out = {'X_init_MNAR': X.double()}
    for p in p_miss: 
        n, d = X.shape
        to_torch = torch.is_tensor(X) ## output a pytorch tensor, or a numpy array
        if not to_torch:
            X = torch.from_numpy(X)
        mask = torch.zeros(n, d).bool() if to_torch else np.zeros((n, d)).astype(bool)
        d_params = max(int(p_params * d), 1) if exclude_inputs else d ## number of variables used as inputs (at least 1)
        d_na = d - d_params if exclude_inputs else d ## number of variables masked with the logistic model
        ### Sample variables that will be parameters for the logistic regression:
        idxs_params = np.random.choice(d, d_params, replace=False) if exclude_inputs else np.arange(d)
        idxs_nas = np.array([i for i in range(d) if i not in idxs_params]) if exclude_inputs else np.arange(d)
        ### Other variables will have NA proportions selected by a logistic model
        ### The parameters of this logistic model are random.
        ### Pick coefficients so that W^Tx has unit variance (avoids shrinking)
        len_obs = len(idxs_params)
        len_na = len(idxs_nas)
        coeffs = torch.randn(len_obs, len_na).double()
        Wx = X[:, idxs_params].mm(coeffs)
        coeffs /= torch.std(Wx, 0, keepdim=True)
        coeffs.double()
        ### Pick the intercepts to have a desired amount of missing values
        len_obs, len_na = coeffs.shape
        intercepts = torch.zeros(len_na)
        for j in range(len_na):
            def f(x):
                return torch.sigmoid(X[:, idxs_params].mv(coeffs[:, j]) + x).mean().item() - p
            intercepts[j] = optimize.bisect(f, -50, 50)
        ps = torch.sigmoid(X[:, idxs_params].mm(coeffs) + intercepts)
        ber = torch.rand(n, d_na)
        mask[:, idxs_nas] = ber < ps
        ## If the inputs of the logistic model are excluded from MNAR missingness,
        ## mask some values used in the logistic model at random.
        ## This makes the missingness of other variables potentially dependent on masked values
        if exclude_inputs:
            mask[:, idxs_params] = torch.rand(n, d_params) < p
        X_nas = X.clone()
        X_nas[mask.bool()] = np.nan
        model_name = 'Missing'
        mask_name = 'Mask'
        out[model_name] = X_nas
        out[mask_name] = mask
    return out


In [3]:
import tpot2

In [4]:
regression_pipeline_full = tpot2.search_spaces.pipelines.SequentialPipeline([
        tpot2.config.get_search_space("imputers"), 
        tpot2.config.get_search_space("regressors"),
    ])

In [5]:
regression_full = {
                    'scorers':['neg_root_mean_squared_error'],
                    'scorers_weights':[1],
                    'population_size' : 5,
                    'survival_percentage':1, 
                    'initial_population_size' : 5,
                    'generations' : 5, 
                    'n_jobs':5,
                    'cv': sklearn.model_selection.KFold(n_splits=10, shuffle=True, random_state=0),
                    'verbose':5, 
                    'max_time_seconds': 360000,
                    'max_eval_time_seconds':60*10, 
                    'classification' : False,
                    'search_space': regression_pipeline_full,
                    'preprocessing':False,
}

In [6]:
def load_task(base_save_folder, task_id, r_or_c):
    
    cached_data_path = f"{base_save_folder}/{task_id}.pkl"
    print(cached_data_path)
    if os.path.exists(cached_data_path):
        d = pickle.load(open(cached_data_path, "rb"))
        X_train, y_train, X_test, y_test = d['X_train'], d['y_train'], d['X_test'], d['y_test']
    else:
        #kwargs = {'force_refresh_cache': True}
        task = openml.datasets.get_dataset(task_id)
        X, y, _, _  = task.get_data(dataset_format="dataframe")
        print(X)
        print(y)
        if y is None: 
            y = X.iloc[:, -1:]
            X = X.iloc[:, :-1]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
        preprocessing_pipeline = sklearn.pipeline.make_pipeline(
            tpot2.builtin_modules.ColumnSimpleImputer(
                "categorical", strategy='most_frequent'), 
            tpot2.builtin_modules.ColumnSimpleImputer(
                "numeric", strategy='mean'), 
                tpot2.builtin_modules.ColumnOneHotEncoder(
                    "categorical", min_frequency=0.001, handle_unknown="ignore")
            )
        X_train = preprocessing_pipeline.fit_transform(X_train)
        X_test = preprocessing_pipeline.transform(X_test)

        X_train = sklearn.preprocessing.normalize(X_train)
        X_test = sklearn.preprocessing.normalize(X_test)

        if r_or_c =='c':
            le = sklearn.preprocessing.LabelEncoder()
            y_train = le.fit_transform(y_train)
            y_test = le.transform(y_test)

        d = {"X_train": X_train, "y_train": y_train, "X_test": X_test, "y_test": y_test}
        if not os.path.exists(f"{base_save_folder}"):
            os.makedirs(f"{base_save_folder}")
        with open(cached_data_path, "wb") as f:
            pickle.dump(d, f)

    return X_train, y_train, X_test, y_test

In [7]:
testpipe = regression_pipeline_full.generate().export_pipeline()

In [8]:
X_train, y_train, X_test, y_test = load_task(base_save_folder='../data', task_id=197, r_or_c= 'r')
for level in [0.3]:
        for type_1 in ['MNAR']:
                X_train = pd.DataFrame(X_train)
                X_test = pd.DataFrame(X_test)
                X_train_M, mask_train = add_missing(X_train, add_missing=level, missing_type=type_1)
                X_test_M, mask_test = add_missing(X_test, add_missing=level, missing_type=type_1)
                X_train_n = X_train_M.to_numpy()
                X_test_n = X_test_M.to_numpy()

../data/197.pkl
      lread  lwrite   scall  sread  swrite   fork   exec     rchar     wchar  \
0       6.0     2.0  1036.0  103.0   114.0   1.00   1.00  172076.0  355965.0   
1       1.0     0.0  2165.0  205.0   101.0   0.40   1.20   43107.0   44139.0   
2      62.0    77.0  3806.0  258.0   166.0   1.40   1.40  492142.0  268706.0   
3       5.0     0.0  4721.0  256.0   177.0   0.99   2.58  524787.0  174964.0   
4      42.0    55.0  3949.0  249.0   244.0   2.60   4.60  197289.0  529200.0   
...     ...     ...     ...    ...     ...    ...    ...       ...       ...   
8187   74.0    49.0  2688.0  176.0   103.0  11.00  32.20   57714.0   38484.0   
8188   29.0    40.0  1906.0  118.0    90.0   0.80   2.00    8175.0   27313.0   
8189    3.0     0.0   926.0   90.0    67.0   0.60   1.00    5411.0   19322.0   
8190    4.0     0.0   418.0   30.0    29.0   0.80   1.00    3959.0   10679.0   
8191    5.0     0.0  1888.0  248.0   215.0   6.20   1.80  216420.0   39346.0   

      pgout  ...  pgsca

/var/folders/b8/05y1vdbd3vd52x71qyhklyjw0000gn/T/ipykernel_6124/3242815643.py:10: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  task = openml.datasets.get_dataset(task_id)
/var/folders/b8/05y1vdbd3vd52x71qyhklyjw0000gn/T/ipykernel_6124/518150171.py:6: FutureWarning: Downcasting behavior in Series and DataFrame methods 'where', 'mask', and 'clip' is deprecated. In a future version this will not infer object dtypes or cast all-round floats to integers. Instead call result.infer_objects(copy=False) for object inference, or cast round floats explicitly. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  missing_mask = missing_mask.mask

In [13]:
X_train_M.isna().sum()

0     2215
1     2204
2     2177
3     2249
4     2188
5     2253
6     2171
7     2272
8     2195
9     2156
10    2200
11    2210
12    2194
13    2141
14    2154
15    2174
16    2196
17    2246
18    2262
19    2209
20    2231
dtype: int64

In [14]:
testpipe.fit(X_train_M, y_train)

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('xgbregressor',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_bytree=None, device=None,
                              early_stopping_rounds=None,
                              enable_categorical=False, eval_metric=None,
                              feature_types=None, gamma=0.000308538695,
                              grow_policy=None, importance_type=None,
                              interaction_constraints=None,
                              learning_rate=0.0164213396083, max_bin=None,
                              max_cat_threshold=None, max_cat_to_onehot=None,
                              max_delta_step=None, max_depth=17,
                              max_leaves=None, min_child_weight=7, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=100, n_jobs=1, nthread=1,
                              num_parallel_tree=None, ...))])

In [15]:
print(testpipe)

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('xgbregressor',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_bytree=None, device=None,
                              early_stopping_rounds=None,
                              enable_categorical=False, eval_metric=None,
                              feature_types=None, gamma=0.000308538695,
                              grow_policy=None, importance_type=None,
                              interaction_constraints=None,
                              learning_rate=0.0164213396083, max_bin=None,
                              max_cat_threshold=None, max_cat_to_onehot=None,
                              max_delta_step=None, max_depth=17,
                              max_leaves=None, min_child_weight=7, missing=nan,
                              monotone_constraints=None, multi_

In [ ]:
dataset_file = '/Users/gabrielketron/tpot2_addimputers/tpot2/ImputerExperiments/data/Spam.csv'
#%% System Parameters
# 1. Mini batch size
mb_size = 128
# 2. Missing rate
p_miss = 0.2
# 3. Hint rate
p_hint = 0.9
# 4. Loss Hyperparameters
alpha = 10
# 5. Train Rate
train_rate = 0.8

#%% Data

# Data generation
Data = np.loadtxt(dataset_file, delimiter=",",skiprows=1)
X = pd.DataFrame(Data)

X_M, mask = add_missing(X)

no, dim = X_M.shape
idx = np.random.permutation(no)

Train_No = int(no * 0.7)
Test_No = no - Train_No
    
# Train / Test Features
trainX = X_M.iloc[:Train_No]
testX = X_M.iloc[Train_No:]

# Train / Test Missing Indicators
trainM = mask.iloc[:Train_No]
testM = mask.iloc[Train_No:]







/var/folders/b8/05y1vdbd3vd52x71qyhklyjw0000gn/T/ipykernel_5982/518150171.py:6: FutureWarning: Downcasting behavior in Series and DataFrame methods 'where', 'mask', and 'clip' is deprecated. In a future version this will not infer object dtypes or cast all-round floats to integers. Instead call result.infer_objects(copy=False) for object inference, or cast round floats explicitly. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  missing_mask = missing_mask.mask(missing_mask.notna(), False)


In [ ]:
class GainImputer(BaseEstimator, TransformerMixin):
    """
    Base class for all imputers.
    It adds automatically support for `add_indicator`.
    """

    def __init__(self, 
                 batch_size=128, 
                 hint_rate=0.9, 
                 alpha=100, 
                 iterations=10000,
                 train_rate = 0.8,
                 learning_rate = 0.001,
                 p_miss = 0.2,
                 random_state=None):
        self.batch_size = batch_size
        self.hint_rate = hint_rate
        self.alpha = alpha
        self.iterations = iterations
        self.train_rate = train_rate
        self.learning_rate = learning_rate
        self.p_miss = p_miss
        self.random_state = random_state
        self.device = (
            "cuda"
            if torch.cuda.is_available()
            else "mps"
            if torch.backends.mps.is_available()
            else "cpu"
            )
        torch.set_default_device(self.device)
        torch.set_default_dtype(torch.float32)
        torch.set_grad_enabled(True)
        if random_state is not None:
            torch.manual_seed(self.random_state)

    def fit(self, X, y=None):
        self.fit_transform(X)
        return self

    def transform(self, X, y = None):
        
        self.modelG.load_state_dict(self._Gen_params)

        if hasattr(X, 'dtypes'):
            X = X.to_numpy()
        #define mask matrix
        X_mask = 1 - np.isnan(X)
        #get dimensions
        no, self.dim = X.shape
        self.int_dim = int(self.dim)
        #normalize the original data, and save parameters for renormalization
        norm_data = X.copy()
        min_val = np.zeros(self.dim)
        max_val = np.zeros(self.dim)
        for i in range(self.dim):
            min_val[i] = np.nanmin(norm_data[i])
            norm_data[:, i] -= np.nanmin(norm_data[:, i])
            max_val[i] = np.nanmax(norm_data[i])
            norm_data[:, i] /= (np.nanmax(norm_data[:, i]) + 1e-06)
        norm_parameters = {'min_val': min_val, 'max_val': max_val}
        norm_data_filled = np.nan_to_num(norm_data, 0)
        p_miss_vec = self.p_miss * np.ones((self.dim,1)) 
        Missing = np.zeros((no,self.dim))
        for i in range(self.dim):
            A = np.random.uniform(0., 1., size = [len(norm_data_filled),])
            B = A > p_miss_vec[i]
            Missing[:,i] = 1.*B

        Z_mb = self._sample_Z(no, self.dim)
        M_mb = Missing
        X_mb = norm_data_filled

        New_X_mb = M_mb * X_mb + (1-M_mb) * Z_mb

        X_mb = torch.tensor(X_mb, dtype=torch.float32)
        New_X_mb = torch.tensor(New_X_mb, dtype=torch.float32)
        M_mb = torch.tensor(M_mb, dtype=torch.float32)

        G_sample = self.modelG(X_mb, New_X_mb, M_mb)
        mse_loss = torch.nn.MSELoss(reduction='mean')
        mse_final = mse_loss((1-M_mb)*X_mb, (1-M_mb)*G_sample)/(1-M_mb).sum()
        print('Transform RMSE: ' + str(np.sqrt(mse_final.item())))

        imputed_data = M_mb * X_mb + (1-M_mb) * G_sample
        imputed_data = imputed_data.cpu().detach().numpy()
        _, dim = imputed_data.shape
        renorm_data = imputed_data.copy()
        for i in range(dim):
            renorm_data[:,i] = renorm_data[:,i] * (max_val[i] + 1e-6)   
            renorm_data[:,i] = renorm_data[:,i] + min_val[i]
        for i in range(dim):
            temp = X[~np.isnan(X[:, i]), i]
            # Only for the categorical variable
            if len(np.unique(temp)) < 20:
                renorm_data[:, i] = np.round(renorm_data[:, i])
        return renorm_data
        
    def fit_transform(self, X, y=None):
        if hasattr(X, 'dtypes'):
            X = X.to_numpy()
        #define mask matrix
        X_mask = 1 - np.isnan(X)
        #get dimensions
        no, self.dim = X.shape
        self.int_dim = int(self.dim)
        #normalize the original data, and save parameters for renormalization
        norm_data = X.copy()
        min_val = np.zeros(self.dim)
        max_val = np.zeros(self.dim)
        for i in range(self.dim):
            min_val[i] = np.nanmin(norm_data[i])
            norm_data[:, i] -= np.nanmin(norm_data[:, i])
            max_val[i] = np.nanmax(norm_data[i])
            norm_data[:, i] /= (np.nanmax(norm_data[:, i]) + 1e-06)
        norm_parameters = {'min_val': min_val, 'max_val': max_val}
        norm_data_filled = np.nan_to_num(norm_data, 0)
        p_miss_vec = self.p_miss * np.ones((self.dim,1)) 
        Missing = np.zeros((no,self.dim))
        for i in range(self.dim):
            A = np.random.uniform(0., 1., size = [len(norm_data_filled),])
            B = A > p_miss_vec[i]
            Missing[:,i] = 1.*B
        #internal test-train split
        # Train / Test Missing Indicators
        #model training
        self.modelD = self.Discriminator(GainImputer=self)
        self.modelG = self.Generator(GainImputer=self)

        optimizer_D = torch.optim.Adam(self.modelD.parameters(), 
                                       lr = self.learning_rate)
        optimizer_G = torch.optim.Adam(self.modelG.parameters(), 
                                       lr = self.learning_rate)
        
        bce_loss = torch.nn.BCEWithLogitsLoss(reduction='mean')
        mse_loss = torch.nn.MSELoss(reduction='mean')

        for it in range(self.iterations):
            mb_idx = self._sample_index(no, self.batch_size)
            X_mb = norm_data_filled[mb_idx,:]
            Z_mb = self._sample_Z(self.batch_size, self.dim)

            M_mb = Missing[mb_idx, :]
            H_mb1 = self._sample_M(self.batch_size, self.dim, 1-self.hint_rate)
            H_mb = M_mb*H_mb1 + 0.5*(1-H_mb1)

            New_X_mb = M_mb * X_mb + (1-M_mb)*Z_mb #introduce missing data

            X_mb = torch.tensor(X_mb, dtype=torch.float32)
            New_X_mb = torch.tensor(New_X_mb, dtype=torch.float32)
            Z_mb = torch.tensor(Z_mb, dtype=torch.float32)
            M_mb = torch.tensor(M_mb, dtype=torch.float32)
            H_mb = torch.tensor(H_mb, dtype=torch.float32)

            #Train Discriminator
            G_sample = self.modelG(X_mb, New_X_mb, M_mb)
            D_prob = self.modelD(X_mb, M_mb, G_sample, H_mb)
            D_loss = bce_loss(D_prob, M_mb)

            D_loss.backward()
            optimizer_D.step()
            optimizer_D.zero_grad()

            #Train Generator
            G_sample = self.modelG(X_mb, New_X_mb, M_mb)
            D_prob = self.modelD(X_mb, M_mb, G_sample, H_mb)
            D_prob.cpu().detach()
            G_loss1 = ((1-M_mb)*(torch.sigmoid(D_prob)+1e-8).log()).mean()/(1-M_mb).sum()
            G_mse_loss = mse_loss(M_mb*X_mb, M_mb*G_sample)/M_mb.sum()
            G_loss = G_loss1 + self.alpha*G_mse_loss

            G_loss.backward()
            optimizer_G.step()
            optimizer_G.zero_grad()

            G_mse_test = mse_loss((1-M_mb)*X_mb, (1-M_mb)*G_sample)/(1-M_mb).sum()

            if it % 100 == 0:
                print('Iter: {}'.format(it))
                print('D_loss: {:.4}'.format(D_loss))
                print('Train_loss: {:.4}'.format(G_mse_loss))
                print()
        self._Gen_params = self.modelG.state_dict()

        Z_mb = self._sample_Z(no, self.dim) 
        M_mb = Missing
        X_mb = norm_data_filled
   
        New_X_mb = M_mb * X_mb + (1-M_mb) * Z_mb

        X_mb = torch.tensor(X_mb, dtype=torch.float32)
        New_X_mb = torch.tensor(New_X_mb, dtype=torch.float32)
        M_mb = torch.tensor(M_mb, dtype=torch.float32)

        G_sample = self.modelG(X_mb, New_X_mb, M_mb)
        mse_final = mse_loss((1-M_mb)*X_mb, (1-M_mb)*G_sample)/(1-M_mb).sum()
        print('Final Train RMSE: ' + str(np.sqrt(mse_final.item())))

        imputed_data = M_mb * X_mb + (1-M_mb) * G_sample
        imputed_data = imputed_data.cpu().detach().numpy()
        _, dim = imputed_data.shape
        renorm_data = imputed_data.copy()
        for i in range(dim):
            renorm_data[:,i] = renorm_data[:,i] * (max_val[i] + 1e-6)   
            renorm_data[:,i] = renorm_data[:,i] + min_val[i]
        for i in range(dim):
            temp = X[~np.isnan(X[:, i]), i]
            # Only for the categorical variable
            if len(np.unique(temp)) < 20:
                renorm_data[:, i] = np.round(renorm_data[:, i])
        return renorm_data
    
    def _sample_M(self, rows, cols, p):
        '''Sample binary random variables.
        Args:
            - p: probability of 1
            - rows: the number of rows
            - cols: the number of columns
        Returns:
            - binary_random_matrix: generated binary random matrix.
        '''
        unif_random_matrix = np.random.uniform(0., 1., size = [rows, cols])
        binary_random_matrix = unif_random_matrix > p
        return 1.*binary_random_matrix

    def _sample_Z(self, rows, cols):
        '''Sample uniform random variables.
        Args:
            - rows: the number of rows
            - cols: the number of columns
        Returns:
            - uniform_random_matrix: generated uniform random matrix.
        '''
        return np.random.uniform(0., 1., size = [rows, cols])       

    def _sample_index(self, rows, batch_size):
        '''Sample index of the mini-batch.
        Args:
            - total: total number of samples (rows)
            - batch_size: batch size
        Returns:
            - batch_idx: batch index
        '''
        total_idx = np.random.permutation(rows)
        batch_idx = total_idx[:batch_size]
        return batch_idx
    
    class Generator(torch.nn.Module):
        def __init__(self, GainImputer):
            super(GainImputer.Generator, self).__init__()
            self.G1 = torch.nn.Linear(GainImputer.dim*2,GainImputer.int_dim)
            self.G2 = torch.nn.Linear(GainImputer.int_dim,GainImputer.int_dim)
            self.G3 = torch.nn.Linear(GainImputer.int_dim,GainImputer.dim)
            self.relu = torch.nn.ReLU()
            self.sigmoid = torch.nn.Sigmoid()
            self.init_weight()

        def init_weight(self):
            layers = [self.G1, self.G2, self.G3]
            [torch.nn.init.xavier_normal_(layer.weight) for layer in layers]

        def forward(self, X: torch.float32, Z: torch.float32, M: torch.float32):
            input = M * X + (1-M)*Z
            input = torch.cat([input, M], dim=1)
            out = self.relu(self.G1(input))
            out = self.relu(self.G2(out))
            out = self.sigmoid(self.G3(out))
            return out
        
    class Discriminator(torch.nn.Module):
        def __init__(self, GainImputer):
            super(GainImputer.Discriminator, self).__init__()
            self.D1 = torch.nn.Linear(GainImputer.dim*2,GainImputer.int_dim)
            self.D2 = torch.nn.Linear(GainImputer.int_dim,GainImputer.int_dim)
            self.D3 = torch.nn.Linear(GainImputer.int_dim,GainImputer.dim)
            self.relu = torch.nn.ReLU()
            self.sigmoid = torch.nn.Sigmoid()
            self.init_weight()
        
        def init_weight(self):
            layers = [self.D1, self.D2, self.D3]
            [torch.nn.init.xavier_normal_(layer.weight) for layer in layers]
        
        def forward(self, X, M, G, H):
            input = M * X + (1-M)*G
            input = torch.cat([input, H], dim=1)
            out = self.relu(self.D1(input))
            out = self.relu(self.D2(out))
            out = self.D3(out)
            return out


In [ ]:
gain = GainImputer(batch_size=128, hint_rate=0.9, alpha=10, iterations=1000)

imputed_train = gain.fit_transform(trainX)
imputed_test = gain.transform(testX)


Iter: 0
D_loss: 0.6997
Train_loss: 3.186e-05

Iter: 100
D_loss: 0.4352
Train_loss: 9.931e-05

Iter: 200
D_loss: 0.3231
Train_loss: 0.0001087

Iter: 300
D_loss: 0.2233
Train_loss: 0.0001086

Iter: 400
D_loss: 0.1838
Train_loss: 0.0001096

Iter: 500
D_loss: 0.145
Train_loss: 0.0001103

Iter: 600
D_loss: 0.1103
Train_loss: 0.0001093

Iter: 700
D_loss: 0.09513
Train_loss: 0.0001099

Iter: 800
D_loss: 0.07875
Train_loss: 0.00011

Iter: 900
D_loss: 0.07089
Train_loss: 0.00011

Final Train RMSE: 0.002079686549113976
Transform RMSE: 0.00319132059301811


In [ ]:
def rmse_loss(ori_data, imputed_data, data_m):
        '''Compute RMSE loss between ori_data and imputed_data
        Args:
            - ori_data: original data without missing values
            - imputed_data: imputed data
            - data_m: indicator matrix for missingness
        Returns:
            - rmse: Root Mean Squared Error
        '''
        #ori_data, norm_parameters = normalization(ori_data)
        #imputed_data, _ = normalization(imputed_data, norm_parameters)
        # Only for missing values
        nominator = np.sum(((1-data_m) * ori_data - (1-data_m) * imputed_data)**2)
        denominator = np.sum(1-data_m)
        rmse = np.sqrt(nominator/float(denominator))
        return rmse


print(f'Train RMSE: {rmse_loss(ori_data=X[:Train_No].to_numpy(), imputed_data=imputed_train, data_m=trainM.to_numpy())}')
print(f'Test RMSE: {rmse_loss(ori_data=X[Train_No:].to_numpy(), imputed_data=imputed_test, data_m=testM.to_numpy())}')

Train RMSE: 209.2995963436833
Test RMSE: 89.76741133216728
